In [1]:
!pip install git+https://github.com/finetuneanon/transformers@gpt-neo-localattention3-rp-b

  Cloning https://github.com/finetuneanon/transformers (to revision gpt-neo-localattention3-rp-b) to /tmp/pip-req-build-noohzkty
  Running command git clone -q https://github.com/finetuneanon/transformers /tmp/pip-req-build-noohzkty
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [2]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!wget http://the-eye.eu/public/AI/carp_seq2seq.zip

--2021-10-15 19:37:35--  http://the-eye.eu/public/AI/carp_seq2seq.zip
Resolving the-eye.eu (the-eye.eu)... 162.213.130.242
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:80... failed: Connection timed out.
Retrying.

--2021-10-15 19:39:46--  (try: 2)  http://the-eye.eu/public/AI/carp_seq2seq.zip
Connecting to the-eye.eu (the-eye.eu)|162.213.130.242|:80... 

In [ ]:
#We're using Neo for this tutorial
from transformers import GPTNeoModel, GPTNeoForCausalLM,\
    GPT2Tokenizer, GPTNeoConfig
import torch

In [ ]:
!unzip carp_seq2seq.zip

In [ ]:
%ls

In [ ]:
import torch
from transformers import GPTNeoForCausalLM, AutoConfig

# GPT-J 6B config
config = AutoConfig.from_pretrained("EleutherAI/gpt-neo-2.7B")
config.attention_layers = ["global"] * 28
config.attention_types = [["global"], 28]
config.num_layers = 28
config.num_heads = 16
config.hidden_size = 256 * config.num_heads
config.vocab_size = 50400
config.rotary = True
config.rotary_dim = 64
config.jax = True

try:
    from collections.abc import MutableMapping
except ImportError:
    from collections import MutableMapping

class Checkpoint(MutableMapping):
    def __init__(self):
        self.checkpoint = torch.load("j6b_ckpt/m.pt", map_location="cpu")
    def __len__(self):
        return len(self.checkpoint)
    def __getitem__(self, key):
        return torch.load(self.checkpoint[key], map_location="cpu")
    def __setitem__(self, key, value):
        return
    def __delitem__(self, key, value):
        return
    def keys(self):
        return self.checkpoint.keys()
    def __iter__(self):
        for key in self.checkpoint:
            yield (key, self.__getitem__(key))
    def __copy__(self):
        return Checkpoint()
    def copy(self):
        return Checkpoint()

model = GPTNeoForCausalLM.from_pretrained(pretrained_model_name_or_path=None, config=config, state_dict=Checkpoint())

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer.add_tokens(["[QUOTE]"])

In [ ]:
%%capture
model = model.half().to("cuda")

In [ ]:
start_story = "Tom wanted a sandwich."
start_crit = "I think after Tom gets a sandwich, you should express how full he is."
start_example = "Passage: " + start_story + "\nCritique: " + start_crit + "\n<|endoftext|>\n"


story = """Mornings I brough water for Lieutenant Awn to bath in, and dressed her, though the local costume was a good deal less effort than her uniform, and she had stopped wearing any sort of cosmetics two years before, as they were difficult to maintain in the heat."""
input_string = "Passage: " + story + "\nCritique: "

critique_start = "I think before this happens, the character should "

toks = tokenizer.encode_plus(start_example + input_string + critique_start, return_tensors="pt").to("cuda")
quote_idx = len(tokenizer)-1

model.config.max_length=2048
out = model.generate(**toks, num_beams=4, repetition_penalty=1.2, no_repeat_ngram_size=4, early_stopping=True, min_length=len(toks['input_ids'][0])+30, bad_words_ids=[[18559], [59], [quote_idx]])
print(tokenizer.decode(out[0]))